In [0]:
%sql
/*
NoteBook que simula el Job JP_PATRIF_GAR_0500_INS_ARCHIVO Que solamente genera la insercion de datos de hacia CIERREN.TTAFOGRAL_ARCHIVO del ambiente de OCI
para las solicitudes aceptadas de SUBPROCESO 364, 365, 368
*/

In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_proceso": str,
        "sr_subproceso": str,
        "sr_subetapa": str,
        "sr_folio": str,
        "sr_instancia_proceso": str,
        "sr_usuario": str,
        "sr_etapa": str,
        "sr_id_snapshot": str,
        "sr_id_archivo": str,
        "sr_fec_arc": str,
        "sr_paso": str,
        "sr_id_archivo_06": str,
        "sr_id_archivo_09": str
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("GAR")).orderBy("Archivo SQL"))

In [0]:
from datetime import datetime
import pytz

# Inicializa la clase para subir los archivos
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato YYYYMMDD
fecha_actual = datetime.now(mexico_tz).strftime("%Y%m%d")
#fecha timeStamp
fecha_actual_sin_formato = datetime.now(mexico_tz).strftime("%d-%m-%Y %H:%M:%S.%f")

In [0]:
statement_001 = query.get_statement(
"GAR_0500_INS_ARCH_ACEPT.sql",
SR_FOLIO=params.sr_folio,
SR_PROCESO=params.sr_proceso,
SR_SUBPROCESO=params.sr_subproceso,
SR_PREFIJO_ARCH_AG_365=conf.prefijo_arch_ag_365,
SR_FECHA_ACTUAL=fecha_actual,
SR_TIMESTAMP=fecha_actual_sin_formato,
SR_NOM_ARCHIVO_ACEPT_AG_365=conf.nom_arch_acept_ag_365,
SR_PREFIJO_ARCH_TA_364=conf.prefijo_arch_ta_364,
SR_NOM_ARCHIVO_ACEPT_TA_364=conf.nombre_arch_acept_ta_364,
SR_PREFIJO_ARCH_UG_368=conf.prefijo_arch_ug_368,
SR_NOM_ARCHIVO_ACEPT_UG_368=conf.nombre_arch_acept_ug_368,
SR_USUARIO=params.sr_usuario,
SR_EXT_TXT=conf.ext_txt,
SR_ID_ARCHIVO_09=params.sr_id_archivo_09
)

In [0]:
if conf.debug:
   df = db.read_data("default", statement_001)

if conf.debug:
   display(df)


In [0]:
temp_delta = 'DELTA_0500_INS_ARCHIVO_ACEPTADOS_'+ params.sr_subproceso + '_' + params.sr_folio
db.write_delta(temp_delta, db.read_data("default", statement_001), "overwrite")
if conf.debug:
    display(db.read_delta(temp_delta))

In [0]:
from pyspark.sql.functions import col, upper
from pyspark.sql.types import StructType, StructField, StringType, LongType

# 1. Definir la ruta del archivo/directorio
file_path = conf.external_location+conf.err_repo_path # Reemplaza con tu ruta real

# Usar dbutils.fs.ls para obtener los metadatos.
# Si es un solo archivo, la lista contendrá un solo elemento.
# Nota: dbutils.fs.ls() devuelve objetos FileInfo con atributos como 'path', 'name', 'size', 'isDir', etc.
files_list = dbutils.fs.ls(file_path)

   #display(files_list)


# Seleccionar solo las columnas 'nombre' y 'edad'
df_tamaño_arch = spark.createDataFrame(files_list).select("name", "size")

from pyspark.sql.functions import col, upper

# Seleccionar columnas, renombrarlas y aplicar funciones
df_info_arch = df_tamaño_arch.select(
    col("name").alias("FTC_NOMBRE"), # Renombrar la columna
    col("size").alias("FTN_TAM_ARCHIVO")) # Aplicar función (mayúsculas)

    
if conf.debug:
   display(df_info_arch)

In [0]:

#GENERA UN DATAFRAME A PARTIR DE LA TABLA DELTA CREADA UN PASO ANTERIOR
df_archivo = db.read_delta(f"DELTA_0500_INS_ARCHIVO_ACEPTADOS_{params.sr_subproceso}_{params.sr_folio}")\
.select("FTN_ID_ARCHIVO",
        "FTC_NOMBRE",
        #"FTN_TAM_ARCHIVO",
        "FCN_ID_PROCESO",
        "FCN_ID_SUBPROCESO",
        "FTD_FEH_GEN",
        "FTD_FEH_ACT",
        "FTC_USU_ACT",
        "FTD_FEH_ARCHIVO",
        "FCN_ID_ESTATUS_ARCHIVO")

#Crea una vista temporal de los anteriores dataframes
df_archivo.createOrReplaceTempView("archivo")
df_info_arch.createOrReplaceTempView("info_arch")


df_inserta_archivo = spark.sql("""
    SELECT
        A.FTN_ID_ARCHIVO,
        A.FTC_NOMBRE,
        I.FTN_TAM_ARCHIVO,
        A.FCN_ID_PROCESO,
        A.FCN_ID_SUBPROCESO,
        A.FTD_FEH_GEN,
        A.FTD_FEH_ACT,
        A.FTD_FEH_ARCHIVO,
        A.FTC_USU_ACT,
        A.FCN_ID_ESTATUS_ARCHIVO
    FROM
        archivo A
    INNER JOIN
        info_arch I ON A.FTC_NOMBRE = I.FTC_NOMBRE
""")

if conf.debug:
   display(df_inserta_archivo)

In [0]:
#Carga los datos del primer dataframe a la tabla auxiliar CIERREN_DATAUX.TTAFOGRAL_ARCHIVO_AUX
db.write_data(df_inserta_archivo, "CIERREN_DATAUX.TTAFOGRAL_ARCHIVO_AUX", "default", "append")



In [0]:
statement_002 = query.get_statement(
"GAR_0500_INS_ARCH_ACEPT_MERGE.sql",
SR_FOLIO=params.sr_folio,
SR_ID_ARCHIVO_09=params.sr_id_archivo_09
)

In [0]:
execution = db.execute_oci_dml(
    statement=statement_002, async_mode=False
)

In [0]:
statement_003 = query.get_statement(
"GAR_0500_INS_ARCH_ACEPT_DELETE_AUX.sql",
SR_ID_ARCHIVO_09=params.sr_id_archivo_09
)


In [0]:
execution = db.execute_oci_dml(
    statement=statement_003, async_mode=False
)

In [0]:
CleanUpManager.cleanup_notebook(locals())